To run selenium first you have to install correct driver. Firefox require gecko:

https://github.com/mozilla/geckodriver/releases

Folder containing driver have to be added to PATH

In [83]:
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import StaleElementReferenceException

In [45]:
user = ''
passwd = ''

book_title = "Black Rednecks and White Liberals"


# Open, redirect and sign in

To do anything with audible you have to sign in first

In [52]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Firefox()
driver.get("http://www.audible.com")

# check are we on audible uk
assert 'Audible UK' in driver.title

# go to audible com
element = driver.find_element_by_link_text("go to audible.com")

assert element.tag_name=='a'
assert element.text=='go to audible.com' 
assert 'audible.com' in element.get_attribute("href")

element.click()

# check are we on audible com
assert 'Audible.com' in driver.title

# Sign In
element = driver.find_element_by_link_text("Sign In")

assert element.tag_name=='a'
assert element.text=='Sign In' 
assert 'www.amazon.com/ap/signin' in element.get_attribute("href")

element.click()

# check are we on audible com
assert 'Amazon Sign In' in driver.title

email_element = driver.find_element_by_name("email")
email_element.send_keys(user)
password_element = driver.find_element_by_name("password")
password_element.send_keys(passwd)

driver.find_element_by_id('signInSubmit').click()

assert 'Download Audiobooks with Audible.com' in driver.title


# Search

## Search for book

We should be able to search from any page

In [75]:
# book_title = "Black Rednecks and White Liberals"
book_title = "virus"

# global-search-legacy
search_element = driver.find_element_by_id('global-search-legacy')
search_element.send_keys(book_title, Keys.ENTER)

##### Problem with checking on which page we are now
# don't work -> assert 'Audiobooks matching keywords' in driver.title

# wait until we get results
WebDriverWait(driver, 10).until(
    EC.text_to_be_present_in_element(
        (By.CSS_SELECTOR, "body div.desktop div#top-3 h1.bc-heading"), 
        'Showing results for "'+book_title+'"'
    ) 
)


True

## Iterate over search results

Then we can iterate over results to look at books, authors...

In [76]:
# Iteration over search results

# every entry is a one row with all book infromations
entries = driver.find_elements_by_css_selector(
    "body div.desktop > div.bc-container > div.bc-row-responsive > div.bc-col-responsive.bc-col-9 li.bc-list-item.productListItem"
)

for entry in entries:
    
    # left side of book row - description
    description_element = entry.find_element_by_css_selector('div.bc-row-responsive  > div.bc-col-responsive.bc-spacing-top-none.bc-col-8')
    
    # right side of book row - price, but and wish list buttons
    price_element = entry.find_element_by_css_selector('div.bc-row-responsive  > div.bc-col-responsive.bc-col-4')
    
    # title line with book link
    title_element = description_element.find_element_by_css_selector("div.bc-col-responsive.bc-col-6 li.bc-list-item a")
    title = title_element.text
    book_link = title_element.get_attribute("href").split('?')[0]
    
    print title + " ===> " + book_link
    
    # authors - book may have multiple authors
    authors_element = description_element.find_elements_by_css_selector("div.bc-col-responsive.bc-col-6 li.bc-list-item.authorLabel > span > a")
    
    for author_element in authors_element:
        author = author_element.text
        
        # amazon may have page for specific author or provide search link
        author_link = author_element.get_attribute("href")
        if(author_link.startswith('https://www.audible.com/search')):
            author_link = author_link.split('&')[0]
        else:
            author_link = author_link.split('?')[0]
    
        print author + " -> " + author_link
        
    print
    

    


Hot Zone ===> https://www.audible.com/pd/Mysteries-Thrillers/Hot-Zone-Audiobook/B075THYBJJ
Steven Konkoly -> https://www.audible.com/author/Steven-Konkoly/B0047MCYNI

Fire Storm: A Post-Apocalyptic Pandemic Thriller ===> https://www.audible.com/pd/Sci-Fi-Fantasy/Fire-Storm-A-Post-Apocalyptic-Pandemic-Thriller-Audiobook/B07BHWZQ78
Steven Konkoly -> https://www.audible.com/author/Steven-Konkoly/B0047MCYNI

Kill Box: A Post-Apocalyptic Pandemic Thriller ===> https://www.audible.com/pd/Sci-Fi-Fantasy/Kill-Box-A-Post-Apocalyptic-Pandemic-Thriller-Audiobook/B079GQQVZQ
Steven Konkoly -> https://www.audible.com/author/Steven-Konkoly/B0047MCYNI

ARES Virus: Phoenix Rising ===> https://www.audible.com/pd/Sci-Fi-Fantasy/ARES-Virus-Phoenix-Rising-Audiobook/B079H1693G
John O'Brien -> https://www.audible.com/author/John-OBrien/B005IDEPP0

Rabid ===> https://www.audible.com/pd/History/Rabid-Audiobook/B008K4B6JC
Bill Wasik -> https://www.audible.com/author/Bill-Wasik/B001JP3YMG
Monica Murphy -> https:

## Go to next page of search results

In [30]:
# pagination

pagination_button_elements = driver.find_elements_by_css_selector(
    "body div.desktop > div.bc-container > div.bc-row-responsive > div.bc-col-responsive.bc-col-9 form.paginationElements button"
)

if len(pagination_button_elements)==2:
    next_page_button = pagination_button_elements[1]

    if next_page_button.get_attribute("disabled")=='true':
        print "Last page"
    else:
        next_page_button.click()
        
else:
    print "No pages"

Last page


In [31]:
# driver.close()

# Book
## Open book page using url

In [92]:
# open book page

#driver.get("https://www.audible.com/pd/Mysteries-Thrillers/Hot-Zone-Audiobook/B075THYBJJ")
driver.get("https://www.audible.com/pd/History/Rabid-Audiobook/B008K4B6JC")


## Load all book's reviews

Click "Show More" button in review section to load all reviews.

In [94]:
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import StaleElementReferenceException

import time

# load all reviews
while True:
    try:
        reviews_section_element = driver.find_element_by_css_selector("div.listReviewsTabUS")
        show_more_button = reviews_section_element.find_element_by_css_selector("div#pagingButtonsUS button")
        show_more_button.click()
    except NoSuchElementException:
        print "No more reviews"
        break;
    except ElementClickInterceptedException:
        print "Problem with click. Waiting 3s..."
        time.sleep(3)
    except StaleElementReferenceException:
        print "Problem with button not connected to DOM tree. Waiting 3s..."
        time.sleep(3)


Problem with click. Waiting 3s...
Problem with click. Waiting 3s...
Problem with click. Waiting 3s...
Problem with click. Waiting 3s...
Problem with button not connected to DOM tree. Waiting 3s...
No more reviews


## Extract reviews content

Extract content of all reviews loaded currently on page.

In [95]:
# get reviews

review_elements = driver.find_elements_by_css_selector("div.listReviewsTabUS > div.bc-row-responsive.bc-spacing-top-medium.bc-spacing-medium")

print len(review_elements)

for review_element in review_elements:
    # left bottom side of review section
    reviewer_element = review_element.find_element_by_css_selector("div.bc-col-responsive.bc-col-3 > div.bc-row-responsive.bc-spacing-top-small > div.bc-col-responsive.bc-col-9 a.bc-link.bc-color-link")
    reviewer_name = reviewer_element.text
    reviewer_url = reviewer_element.get_attribute("href").split('?')[0]
    
    # left top side of review section
    rating_elements = review_element.find_elements_by_css_selector("div.bc-col-responsive.bc-col-3 > span li")
    
    overall_rating = len(rating_elements[0].find_elements_by_css_selector("i.bc-color-progress"))
    performance_rating = len(rating_elements[1].find_elements_by_css_selector("i.bc-color-progress"))
    story_rating = len(rating_elements[1].find_elements_by_css_selector("i.bc-color-progress"))
    
    print reviewer_name + " -> "+reviewer_url
    print "overall: "+str(overall_rating)
    print "performance: "+str(performance_rating)
    print "story: "+str(story_rating)
    print
    


88
Sparkly -> https://www.audible.com/listener/A2THA844IKD3KZ
overall: 5
performance: 4
story: 4

Cynthia -> https://www.audible.com/listener/A8P2L9UWYOZUM
overall: 4
performance: 4
story: 4

Jean -> https://www.audible.com/listener/AOM8H2MJBGIMK
overall: 4
performance: 5
story: 5

Linda Trader -> https://www.audible.com/listener/A28VITZJXK75FU
overall: 4
performance: 4
story: 4

Carolyn -> https://www.audible.com/listener/A3QN0WX60AWWTQ
overall: 5
performance: 4
story: 4

Sean -> https://www.audible.com/listener/A1VUHO8W3XZ6CH
overall: 5
performance: 5
story: 5

Jan -> https://www.audible.com/listener/A2G3JK3KXYH0FD
overall: 2
performance: 3
story: 3

Douglas -> https://www.audible.com/listener/A3MNUBIY41OCTL
overall: 5
performance: 5
story: 5

Atila -> https://www.audible.com/listener/A78Q3965OOORE
overall: 3
performance: 4
story: 4

Mary Hirsch -> https://www.audible.com/listener/AAAH7JYY7MJER
overall: 5
performance: 5
story: 5

David -> https://www.audible.com/listener/A1TM0DVJIF5U

Tyler C -> https://www.audible.com/listener/A2X653IC0RN011
overall: 4
performance: 5
story: 5

Ellenaeddy -> https://www.audible.com/listener/A3H5SYPXFJ51KM
overall: 3
performance: 2
story: 2



# User
## Open user page using url

In [96]:
driver.get("https://www.audible.com/listener/A2THA844IKD3KZ")

## Load all user's reviews

In [99]:
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import ElementNotInteractableException

import time

# load all reviews
while True:
    try:
        show_more_button = driver.find_element_by_css_selector("div#profile_content > div#adbl-reviews-by-me-cont > div.more-cont > a.adbl-profile-more.adbl-more-link")    
        show_more_button.click()
    except NoSuchElementException:
        print "No more reviews"
        break;
    except ElementClickInterceptedException:
        print "Problem with click. Waiting 3s..."
        time.sleep(3)
    except StaleElementReferenceException:
        print "Problem with button not connected to DOM tree. Waiting 3s..."
        time.sleep(3)
    except ElementNotInteractableException:
        print "Problem with button. Waiting 3s..."
        time.sleep(3)

Problem with button. Waiting 3s...
Problem with button. Waiting 3s...
Problem with button. Waiting 3s...
No more reviews


## Extract reviews
Extract content of all reviews loaded on user's page

In [103]:
review_elements = driver.find_elements_by_css_selector("div#profile_content > div#adbl-reviews-by-me-cont li.adbl-review")

print len(review_elements)

for review_element in review_elements:
    book_element = review_element.find_element_by_css_selector("div.adbl-prod-title > a")
    title = book_element.text
    book_link = book_element.get_attribute("href").split('ref')[0][:-1]
    
    print title + " -> " + book_link


45
The Great Passage -> http://www.audible.com/pd/Fiction/The-Great-Passage-Audiobook/B071YFQKZ7
The Tattoo Murder Case -> http://www.audible.com/pd/Mysteries-Thrillers/The-Tattoo-Murder-Case-Audiobook/B004XCGTRO
The Gentle Art of Swedish Death Cleaning: How to Free Yourself and Your Family from a Lifetime of Clutter -> http://www.audible.com/pd/Self-Development/The-Gentle-Art-of-Swedish-Death-Cleaning-Audiobook/B077H3JCJY
The Three-Body Problem -> http://www.audible.com/pd/Sci-Fi-Fantasy/The-Three-Body-Problem-Audiobook/B00P0277C2
Tinker, Tailor, Soldier, Spy: A George Smiley Novel -> http://www.audible.com/pd/Mysteries-Thrillers/Tinker-Tailor-Soldier-Spy-Audiobook/B006IX26WI
The Hoarder in You: How to Live a Happier, Healthier, Uncluttered Life -> http://www.audible.com/pd/Self-Development/The-Hoarder-in-You-Audiobook/B006NT3P5O
How I Helped O. J. Get Away with Murder: The Shocking Inside Story -> http://www.audible.com/pd/Bios-Memoirs/How-I-Helped-O-J-Get-Away-with-Murder-Audiobook/